[<img src="../header.svg">](../index.ipynb)

---

# Application: Simulation of Drug Transport across a Virtual Skin Membrane

### Preliminaries

In [1]:
import modsimtools
from modsimtools import *

['/Users/anaegel/Software/ug4-git/apps/modsim-course-python/content/skin', '/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.3/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/opt/homebrew/lib/python3.11/site-packages', '/opt/homebrew/Cellar/pygments/2.14.0/libexec/lib/python3.11/site-packages', '/Users/anaegel/Software/ug4-git/lib/', '/Users/anaegel/Software/ug4-git/bin/plugins/ug4py']


In [ ]:
%%writefile modsimtools.py

import os
env_ug4_root = os.environ['UG4_ROOT']
import sys
sys.path.append(env_ug4_root+'/lib/')
sys.path.append(env_ug4_root+'/bin/plugins/ug4py')

### Create some widgets

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

s = widgets.Select(
    options=['skin2d-aniso.ugx', 'skin2d-aniso2.ugx', 'skin2d-aniso3.ugx'],
    value='skin2d-aniso.ugx', rows=1,
    description='Geometry:', disabled=False)

r = widgets.BoundedIntText(
    value=2, min=0, max=5,
    step=1, description='Refinements:',
    disabled=False)

display(s)
display(r)

Select(description='Geometry:', options=('skin2d-aniso.ugx', 'skin2d-aniso2.ugx', 'skin2d-aniso3.ugx'), rows=1…

BoundedIntText(value=2, description='Refinements:', max=5)

## Solve problem

In [ ]:
%%writefile -a modsimtools.py

import ug4py as ug4
import pylimex as limex
import pyconvectiondiffusion as cd
import pysuperlu as slu

In [3]:
# Setup:
# defining needed subsets, gird and number of refinements
requiredSubsets = ["LIP", "COR", "BOTTOM_SC", "TOP_SC"] # defining subsets
gridName = 'skin2d-aniso.ugx'  # Grid created with ProMesh
numRefs = 1  # Number of Refinement steps on given grid

In [ ]:
%%writefile -a modsimtools.py

def CreateDomain(gridName, numRefs, requiredSubsets):
    # Choosing a domain object
    # (either 1d, 2d or 3d suffix)
    dom = ug4.Domain2d()

    # Loading the given grid into the domain
    print("Loading Domain {gridName}...")
    ug4.LoadDomain(dom, gridName)
    print("Domain loaded.")
    
    
    # Optional: Refining the grid
    if numRefs > 0:
        print("Refining ...")
        refiner = ug4.GlobalDomainRefiner(dom)
        for i in range(numRefs):
            ug4.TerminateAbortedRun()
            refiner.refine()
            print("Refining step {i} ...")

        print("Refining done")

    # checking if geometry has the needed subsets of the probelm
    sh = dom.subset_handler()
    for e in requiredSubsets:
        if sh.get_subset_index(e) == -1:
            print(f"Domain does not contain subset {e}.")
            sys.exit(1)
    
    return dom

In [4]:
#from modsimtools import CreateDomain
dom = CreateDomain(gridName, numRefs, requiredSubsets)

Loading Domain {gridName}...
Domain loaded.
Refining ...
Refining step {i} ...
Refining done


### Create Approximation space

In [5]:
# Create approximation space which describes the unknowns in the equation
approxSpaceDesc = dict(fct = "u", type = "Lagrange", order = 1)

In [ ]:
%%writefile -a modsimtools.py

def CreateApproximationSpace(dom, approxSpaceDesc):
    approxSpace = ug4.ApproximationSpace2d(dom)
    approxSpace.add_fct(approxSpaceDesc["fct"], approxSpaceDesc["type"], approxSpaceDesc["order"])
    approxSpace.init_levels()
    approxSpace.init_top_surface()
    print("Approximation space:")
    approxSpace.print_statistic()
    return approxSpace

In [6]:
approxSpace = CreateApproximationSpace(dom, approxSpaceDesc)

Approximation space:
| ----------------------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                               |
|  Algebra: Block 1 (divide by 1 for #Index)                                                |
|                                                                                           |
|    GridLevel   |       Domain |       0: LIP |       1: COR | 2: BOTTOM_SC |    3: TOP_SC |
| ----------------------------------------------------------------------------------------- |
| (lev,    0)    |          680 |           32 |          608 |           20 |           20 |
| (lev,    1)    |         2613 |          783 |         1752 |           39 |           39 |
| (lev,    0, g) |          680 |           32 |          608 |           20 |           20 |
| (lev,    1, g) |         2613 |          783 |         1752 |           39 |           39 |
| (surf, top)    |         2613 |      

## Create a convection-diffusion-equation
$$\frac{\partial Ku}{\partial t} + \nabla \cdot [-DK \nabla u] = 0$$
Partition coefficients

In [7]:
K={}
K["COR"]=1e-3
K["LIP"]=1.0

Diffusion coefficients

In [8]:
D={}
D["COR"]=1.0
D["LIP"]=1.0

Creating two instances of a convection diffusion equation

In [ ]:
%%writefile -a modsimtools.py

def CreateDiffusionElemDisc(fname, subdom, mass_scale, diffusion, reaction):
    elemDisc = cd.ConvectionDiffusionFV12d(fname, subdom)
    elemDisc.set_mass_scale(mass_scale)
    elemDisc.set_diffusion(diffusion)
    elemDisc.set_reaction(reaction)
    return elemDisc

In [9]:
elemDisc={}
elemDisc["COR"] = CreateDiffusionElemDisc("u", "COR", K["COR"], K["COR"]*D["COR"], 0.0)
elemDisc["LIP"] = CreateDiffusionElemDisc("u", "LIP", K["LIP"], K["LIP"]*D["LIP"], 0.0)

### Boundary conditions
ug4 separates the boundary value and the discretization
boundary conditions can be enforced through a post-process (dirichlet).
To init at boundary, the value, function name from the Approximationspace
#and the subset name are needed

In [10]:
dirichletBND = ug4.DirichletBoundary2dCPU1()
dirichletBND.add(1.0, "u", "TOP_SC")
dirichletBND.add(0.0, "u", "BOTTOM_SC")

### Global discretization
Create the discretization object which combines all the separate discretizations into one domain discretization.

In [11]:
domainDisc = ug4.DomainDiscretization2dCPU1(approxSpace)
domainDisc.add(elemDisc["COR"])
domainDisc.add(elemDisc["LIP"])
domainDisc.add(dirichletBND)

## Create  solver

In [12]:
lsolver=ug4.LUCPU1()

In [13]:
import pysuperlu as slu
lsolver=slu.SuperLUCPU1()

## Solve transient problem

Solve the transient problem.
Use the Approximationspace to create a vector of unknowns and a vector which contains the right hand side

In [14]:
# Define start time, end time and step size
startTime = 0.0
endTime = 1000.0
dt = 25.0

Create a time discretization (with the theta-scheme).
Requires domain and theta ($\theta$ = 1.0 -> implicit Euler, 
$\theta$ = 0.0 -> explicit Euler )


In [15]:
timeDisc=ug4.ThetaTimeStepCPU1(domainDisc, 1.0) 

Create time integrator (requires solver and step size).

In [21]:
#timeInt = limex.LinearTimeIntegrator2dCPU1(timeDisc)
timeInt = limex.ConstStepLinearTimeIntegrator2dCPU1(timeDisc)
timeInt.set_linear_solver(lsolver)
timeInt.set_time_step(dt)

Create and attach an observer object for vtk output:

In [22]:
# vtk=ug4.VTKOutput2d()
# obs=limex.VTKOutputObserver2dCPU1("SkinData.vtu", vtk)
# timeInt.attach_observer(obs)

# Exporting the result to a vtu-file
# can be visualized in paraview or with a python extension
# We add a corresponding observer to the time integrator.
def MyCallback(usol, step, time, dt) :
    # print(step)
    # print(time)
    # print(dt)
    ug4.WriteGridFunctionToVTK(usol, "SkinDiffusion_"+str(int(step)).zfill(5)+".vtu")
obs =ug4.PythonCallbackObserver2dCPU1(MyCallback) 

timeInt.attach_observer(obs)

Solving the transient problem

In [23]:
usol = ug4.GridFunction2dCPU1(approxSpace)

# Init the vector representing the unknowns with 0 to obtain
# reproducable results
ug4.Interpolate(0.0, usol, "u")

# Exporting the result to a vtu-file
# can be visualized in paraview or with a python extension
ug4.WriteGridFunctionToVTK(usol, "SkinData_t0000")

In [24]:
try:
    timeInt.apply(usol, endTime, usol, startTime)
except Exception as inst:
    print(inst)

+++ Integrating: [	0	, 	1000	] with dt=	25(40 iters)
+++ Const timestep +++1(t=0, dt=25)
+++ Assemble (t=0, dt=25)
after COLAMD_MAIN info 1
+++ Const timestep +++2(t=25, dt=25)
+++ Const timestep +++3(t=50, dt=25)
+++ Const timestep +++4(t=75, dt=25)
+++ Const timestep +++5(t=100, dt=25)
+++ Const timestep +++6(t=125, dt=25)
+++ Const timestep +++7(t=150, dt=25)
+++ Const timestep +++8(t=175, dt=25)
+++ Const timestep +++9(t=200, dt=25)
+++ Const timestep +++10(t=225, dt=25)
+++ Const timestep +++11(t=250, dt=25)
+++ Const timestep +++12(t=275, dt=25)
+++ Const timestep +++13(t=300, dt=25)
+++ Const timestep +++14(t=325, dt=25)
+++ Const timestep +++15(t=350, dt=25)
+++ Const timestep +++16(t=375, dt=25)
+++ Const timestep +++17(t=400, dt=25)
+++ Const timestep +++18(t=425, dt=25)
+++ Const timestep +++19(t=450, dt=25)
+++ Const timestep +++20(t=475, dt=25)
+++ Const timestep +++21(t=500, dt=25)
+++ Const timestep +++22(t=525, dt=25)
+++ Const timestep +++23(t=550, dt=25)
+++ Const tim

## Plotting the result using pyvista

In [25]:
import pyvista
pyvista.start_xvfb()
result = pyvista.read('SkinDiffusion_00040.vtu')

#pyvista.set_jupyter_backend('pythreejs')
pyvista.set_jupyter_backend('trame')

print("Pyvista input: ")
print(result)
result.plot(scalars="u", show_edges=True, cmap='jet')
#scalars="node_value", categories=True

Pyvista input: 
UnstructuredGrid (0x105e41600)
  N Cells:    2508
  N Points:   2613
  X Bounds:   0.000e+00, 3.010e+01
  Y Bounds:   0.000e+00, 1.760e+01
  Z Bounds:   0.000e+00, 0.000e+00
  N Arrays:   1



Widget(value="<iframe src='http://localhost:58967/index.html?ui=P_0x173ab7c10_1&reconnect=auto' style='width: …